# Домашнее задание 2
## Named Entity Recognition and Event Extraction

deadline: 06 ноября 2022, 23:59

В этом домашнем задании вы будете работать с двумя корпусами - LitBank и MultiNERD_EN.
Первый корпус (LitBank) собран из популярных художественных произведений на английском языке и содержит разметку по именованным сущностям и событиям, корпус состоит из 100 текстов по примерно 2000 слов каждый. 

Корпус описан в статьях:
* David Bamman, Sejal Popat, Sheng Shen, An Annotated Dataset of Literary Entities http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/naacl2019_literary_entities.pdf
* Matthew Sims, Jong Ho Park, David Bamman, Literary Event Detection,  http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/acl2019_literary_events.pdf

Корпус доступен в репозитории проекта:  https://github.com/dbamman/litbank

Статья и код, использованный для извлечения именованных сущностей: 
* Meizhi Ju, Makoto Miwa and Sophia Ananiadou, A Neural Layered Model for Nested Named Entity Recognition, https://github.com/meizhiju/layered-bilstm-crf

Структура корпуса устроена так. 
Первый уровень: 
* entities -- разметка по сущностям
* events -- разметка по сущностям


В корпусе используются 6 типов именованных сущностей: PER, LOC, ORG, FAC, GPE, VEH (имена, локации, организации, помещения, топонимы, средства перемещения), допускаются вложенные сущности. 

События выражается одним словом - *триггером*, которое может быть глаголом, прилагательным и существительным. В корпусе описаны события, которые действительно происходят и не имеют гипотетического характера. 
Пример: she *walked* rapidly and resolutely, здесь *walked* -- триггер события. Типы событий не заданы. 


Второй уровень:
* brat -- рабочие файлы инструмента разметки brat, ann-файлы содержат разметку, txt-файлы – сырые тексты 
* tsv -- tsv-файлы содержат разметку в IOB формате,

Второй корпус (MultiNERD_EN) состоит из 164 тысяч предложений по различным тематикам. От предыдущего корпуса он отличается большим числом (15) типов именованных сущностей. Также одной из сущностей является сущность-событие (EVE). Каждая сущность может состоять как из одного, так и нескольких слов.

Корпус доступен в репозитории проекта:  https://github.com/Babelscape/multinerd

В статьях и репозиториях вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и использовать повторно. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 4-х человек.
2. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо в ipython-тетрадке. При этом прикладывать ipynb-файл обязательно. 
43. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
5. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
6. Плагиат и любое недобросовестное цитирование приводит к обнулению оценки. 


## Часть 1. [2 балла] Эксплоративный анализ 
1. [0.4 балла] Найдите топ 10 (по частоте) именованных сущностей каждого из типов для каждого корпуса. Произведите сравнение частотности по совпадающим сущностям при сопоставлении корпусов друг с другом.
2. [0.4 балла] Найдите топ 10 (по частоте) частотных триггеров событий (на основе LitBank). Произведите анализ из пункта 1 для именованной сущности типа EVE (на основе MultiNERD_EN). 
3. [0.6 балла] Кластеризуйте все уникальные триггеры событий (LitBank), используя эмбеддинги слов и любой алгоритм кластеризации (например, аггломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий? 
4. [0.6 балла] Отдельно произведите анализ из п. 3 для сущностей типа EVE (на основе MultiNERD_EN). Сравните результаты из п. 3 и п. 4.

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу LitBank и сравните кластеры триггеров и выделенные темы: есть ли схожие паттерны в тематической модели и в структуре кластеров?

В следующих частях домашнего задания вам понадобится train-test-dev разбиения. Авторы статей предлагают следующую структуру разбиения для корпуса LitBank: обучающее множество – 80 книг, валидационное – 10 книг, тестовое – 10 книг. Предложения из одного источника не должны попадать в разные сегменты разбиения.
Для корпуса MultiNERD_EN произведите стратифицированное разбиение по предложениям в соотношении 80%:10%:10%. Стратификацию производить в отношении пропорции по именованным сущностям (понятно, что с учётом всех вводных идеального разбиения не получится, но старайтесь сохранить распределения по типам сущностей для каждой и подвыборок).


## Часть 2. [3 балла] Извлечение именованных сущностей
Для каждой корпуса необходимо осуществить раздельно следующую процедуру:
1. [0.75 балла] Обучите стандартную модель для извлечения именованных сущностей, CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей. 
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация).
2. [0.75 балла] Замените часть модели на символах и словах (CNN + эмбеддинги словах) на ELMo и / или BERT. Должна получиться модель ELMo / BERT + BiLSTM + CRF. 
3. [1.5 балла] Замените модель последовательности (BiLSTM) на другой слой, например, на Transformer. Должна получиться модель CNN  + Transformer + CRF.
В результате у вас будет 2 набора моделей - проверьте качество извлечения на тестовых данных.
- Примените модели первого корпуса ко всем (!!!) предложениям второго корпуса и наоборот.
- Составьте отчёт по качеству работы моделей в терминах извлечения по типам сущностей, которые встречаются в обоих корпусах. Метрику выберите самостоятельно.
- Проанализируйте ошибки моделей в терминах предсказания сущностям первого корпуса специфических типов для второго корпуса. И наоборот - когда специфическим типам второго корпуса модель, обученная на LitBank, прсваивает одну из своих категорий.

[бонус] Дообучите BERT для извлечения именованных сущностей.

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]

[бонус] Модифицируйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]: вместо эмбеддингов слов используйте ELMo и/или BERT. 

## Часть 3. [2 балла] Извлечение событий 

1. [0.75 балла] Используйте BiLSTM на эмбеддингах слов для извлечения триггеров событий (на основе LitBank). 

2. [0.75 балла] Замените часть модели на словах на ELMo и/или BERT. Должна получиться модель ELMo / BERT + BiLSTM.

3. [0.5 балла] Проверьте "извлекающую" силу модели на данных MultiNERD_EN для событий типа EVE. 

[бонус] Предобучите BiLSTM как языковую модель. Дообучите ее для извлечения триггеров. 

[бонус] Дообучите BERT для извлечения триггеров событий. 

## Часть 4. [2 балла] Одновременное извлечение именованных сущностей и событий 
1. [0.75 балла] Обучите модель для совместного извлечения именованных сущностей и триггеров событий только на основе LitBank (!). У модели должен быть общий энкодер (например, CNN + BiLSMT, ELMo + BiLSTM, BERT + BiLSTM) и два декодера: один отвечает за извлечение именованных сущностей, другой отвечает за извлечение триггеров событий.

2. [0.75 балла] Создайте единую обучающую подвыборку, единую валидационную и единую тестовую на основе разбиений, которые вы произвели в первой части и обучите модель. (Единую - то есть на основе обоих корпусов сразу)

3. [0.5 балла] Сравните предсказательную силу модели из п.1 и модели из п.2 как на совместных подвыборках, так и на раздельных. Проанализируйте полученный результат. Приводит ли обогащение дополнительными данными к улучшению "извлекающей" способности модели?

[бонус] Добавьте в модель механизм внимания, таким способом, который покажется вам разумным.

[бонус] Визуализируйте карты механизма внимания (attention). 

## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

<span style="color:pink">Авторы работы: Токкожин Арсен, Соколов Ян, Екимов Егор, Гвасалия Лукас</span>
# Решение:

## Часть 1. [2 балла] Эксплоративный анализ 
1. [0.4 балла] Найдите топ 10 (по частоте) именованных сущностей каждого из типов для каждого корпуса. Произведите сравнение частотности по совпадающим сущностям при сопоставлении корпусов друг с другом.
2. [0.4 балла] Найдите топ 10 (по частоте) частотных триггеров событий (на основе LitBank). Произведите анализ из пункта 1 для именованной сущности типа EVE (на основе MultiNERD_EN). 
3. [0.6 балла] Кластеризуйте все уникальные триггеры событий (LitBank), используя эмбеддинги слов и любой алгоритм кластеризации (например, аггломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий? 
4. [0.6 балла] Отдельно произведите анализ из п. 3 для сущностей типа EVE (на основе MultiNERD_EN). Сравните результаты из п. 3 и п. 4.

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу LitBank и сравните кластеры триггеров и выделенные темы: есть ли схожие паттерны в тематической модели и в структуре кластеров?

In [4]:
import os
from brat_parser import get_entities_relations_attributes_groups
from collections import Counter

entities_folder_path = "litbank_entities/brat"
events_folder_path = "litbank_events/brat"
multinerd_path = "multinerd_en.tsv"

#### Топ 10 именованных сущностей для корпуса Litbank

In [1]:
entities = ["PER", "FAC", "LOC", "ORG", "GPE", "VEH"]

In [2]:
def count_entities(path_to_folder, entities_names):
    result = {}
    for entity in entities_names:
        result[entity] = Counter()

    for filename in os.listdir(path_to_folder):
        if filename.endswith(".ann"):
            entities, relations, attributes, groups = get_entities_relations_attributes_groups(os.path.join(path_to_folder, filename))
            for item in entities:
                result[entities[item].type][entities[item].text] += 1 

    return result

In [14]:
entities_ans = count_entities(entities_folder_path, entities)
for entity in entities_ans:
    print(entity)
    temp = entities_ans[entity].most_common()[:10]
    for item in temp:
        print(item)
    print("-------------------------------------")

PER
('Mr.', 148)
('Miss', 133)
('Mrs.', 132)
('sir', 50)
('Sir', 45)
('my mother', 40)
('men', 40)
('Cameron', 38)
('his wife', 37)
('Mr', 37)
-------------------------------------
FAC
('home', 65)
('the house', 52)
('there', 39)
('here', 39)
('the room', 34)
('the garden', 23)
('the street', 14)
('the hall', 13)
('the road', 13)
('the place', 12)
-------------------------------------
LOC
('the world', 72)
('the sea', 27)
('the river', 22)
('the country', 20)
('there', 18)
('the earth', 16)
('sea', 16)
('the valley', 13)
('this world', 12)
('the woods', 9)
-------------------------------------
ORG
('the army', 7)
('the Committee of Public Safety', 4)
('the Colonial Office', 4)
('the Church', 4)
('Harvard', 3)
('college', 3)
('the hospital', 2)
('the C.C.H.', 2)
("the Bank of Leichardt 's Land", 2)
('a regiment of regulars', 2)
-------------------------------------
GPE
('London', 40)
('England', 32)
('there', 21)
('the town', 21)
('New York', 16)
('town', 14)
('France', 14)
('Europe', 1

#### Топ 10 именованных сущностей для корпуса MultiNERD_EN

In [27]:
# TO-DO
import pandas as pd
col_names = ["Index", "Token", "Label", "BabelNet ID", "Wikidata ID", "Wikipedia ID", "Wikipedia Title", "Definition", "Image URL", "Smth"]
multinerd_data = pd.read_csv(multinerd_path, sep="\t", names=col_names, index_col=False)

In [21]:
table = pd.read_table(multinerd_path, names=col_names)

In [22]:
table.head()

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL,Smth
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,B-PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...,NaN
3,3,Cameron,I-PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
table.loc[[745]]

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL,Smth
745,54,\tO\t\t\t\t\t\t\t\n55\tBring\tB-MEDIA\tbn:0064...,https://upload.wikimedia.org/wikipedia/en/5/51...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
multinerd_data.head()

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL,Smth
0,0\tCreated\tO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1\tby\tO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2\tJames\tB-PER\tbn:00786804n\tQ42574\t15622\t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3\tCameron\tI-PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4\tand\tO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
print(multinerd_data["Smth"].size)
print(multinerd_data["Smth"].isna().count())

2397395
2397395


видим что последняя колонка полнстью состоит из пропусков, так что ее можно удалить

In [54]:
multinerd_data=multinerd_data.drop(["Smth", "Index"], axis=1)

In [55]:
multinerd_data.head()

,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL
0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN
1,by,O,NaN,NaN,NaN,NaN,NaN,NaN
2,James,B-PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...
3,Cameron,I-PER,NaN,NaN,NaN,NaN,NaN,NaN
4,and,O,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
all_labels = list(multinerd_data["Label"])

In [64]:
multinerd_data.loc[[745]]

,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL
745,\tO\t\t\t\t\t\t\t\n55\tBring\tB-MEDIA\tbn:0064...,https://upload.wikimedia.org/wikipedia/en/5/51...,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
multinerd_data.loc[[748]]

,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL
748,\tO\t\t\t\t\t\t\t\n59\t.\tO\t\t\t\t\t\t\t\n\n0...,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
import csv

new_path = "new.tsv"

with open(new_path) as file:
	
	tsv_file = csv.reader(file, delimiter=r"\t")
	
	cnt = 0
	for line in tsv_file:
		cnt += 1
		# if len(line) != 10:
		# 	# print(line)
		# 	if len(line) != 0:
		# 		print(cnt)
		# 		print(line)
		# if cnt == 775:
		# 	print(line)
		# 	break
		# if cnt == 100:
		# 	break
		print(line)
		if cnt == 20:
			break


['0', 'Created', 'O', '']
['1', 'by', 'O', '']
['2', 'James', 'B-PER', 'bn:00786804n', 'Q42574', '15622', 'James_Cameron', 'James Francis Cameron is a Canadian film director, producer, screenwriter, editor, and environmentalist.', 'https://upload.wikimedia.org/wikipedia/commons/c/cd/JamesCameronHWOFOct2012.jpg', '']
['3', 'Cameron', 'I-PER', '']
['4', 'and', 'O', '']
['5', 'Charles', 'B-PER', 'bn:02320763n', 'Q3666404', '2956611', 'Charles_H._Eglee', 'Charles Hamilton Eglee is an American film and television screenwriter and producer.', 'https://upload.wikimedia.org/wikipedia/commons/c/c0/Charles_Eglee_at_ATX_2014.jpg', '']
['6', 'H.', 'I-PER', '']
['7', 'Eglee', 'I-PER', '']
['8', ',', 'O', '']
['9', 'it', 'O', '']
['10', 'stars', 'O', '']
['11', 'Jessica', 'B-PER', 'bn:02793539n', 'Q44077', '343554', 'Jessica_Alba', 'Jessica Marie Alba is an American actress and businesswoman.', 'https://upload.wikimedia.org/wikipedia/commons/c/c5/Jessica_Alba_2011.jpg', '']
['12', 'Alba', 'I-PER', '

In [26]:
with open(multinerd_path) as file:
	
	tsv_file = csv.reader(file, delimiter="\t")
	
	cnt = 0
	for line in tsv_file:
		cnt += 1
		if len(line) != 0 and len(line) != 10:
			print(cnt)
			print(line)
		if cnt == 1000:
			break

775
['54', '\tO\t\t\t\t\t\t\t\n55\tBring\tB-MEDIA\tbn:00640782n\tQ4968296\t14332339\tBring_Me_Sunshine\tBring Me Sunshine" is a song written in 1966 by the composer Arthur Kent, with lyrics by Sylvia Dee.', 'https://upload.wikimedia.org/wikipedia/en/5/51/Bring_Me_Sunshine_-_Morecambe_%26_Wise.jpg', '']
778
['58', '\tO\t\t\t\t\t\t\t\n59\t.\tO\t\t\t\t\t\t\t\n\n0\tand\tO\t\t\t\t\t\t\t\n1\tAlicia\tB-PER\tbn:17691695n\tQ23771162\t49725365\tAlicia_Previn\tAlicia Previn is an American violinist, songwriter, recording artist and author.\thttps://upload.wikimedia.org/wikipedia/commons/8/8c/AliciaPrevin.jpg\t\n2\tPrevin\tI-PER\t\t\t\t\t\t\t\n3\t.\tO\t\t\t\t\t\t\t\n\n0\tIn\tO\t\t\t\t\t\t\t\n1\t1959\tO\t\t\t\t\t\t\t\n2\t,\tO\t\t\t\t\t\t\t\n3\the\tO\t\t\t\t\t\t\t\n4\tmarried\tO\t\t\t\t\t\t\t\n5\tDory\tB-PER\tbn:00161678n\tQ452252\t1042729\tDory_Previn\tDorothy Dory" Veronica Previn was an American lyricist, singer-songwriter and poet.', 'https://upload.wikimedia.org/wikipedia/commons/2/28/Korrekt%C

In [16]:
pd.read_csv(new_path, sep="\t")

ParserError: Error tokenizing data. C error: Expected 4 fields in line 3, saw 10


In [62]:
all_labels

['O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'B-PER',
 'I-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-TIME',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-EVE',
 'I-EVE',
 'I-EVE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-EVE',
 'I-EVE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 

In [61]:
cnt = Counter()
index = 0
indexes = list()
for label in all_labels:
    if label != "O" and label.endswith(".jpg"):
        indexes.append(index)
    cnt[label] += 1
    index += 1
# cnt
indexes

AttributeError: 'float' object has no attribute 'endswith'

#### Топ 10 частотных триггеров событий для корпуса Litbank

In [13]:
def count_events(path_to_folder):
    cnt = Counter()

    for filename in os.listdir(path_to_folder):
        if filename.endswith(".ann"):
            entities, relations, attributes, groups = get_entities_relations_attributes_groups(os.path.join(path_to_folder, filename))
            for item in entities:
                cnt[entities[item].text] += 1 

    return cnt

In [14]:
count_events(events_folder_path).most_common()[:10]

[('said', 464),
 ('came', 95),
 ('looked', 92),
 ('went', 92),
 ('asked', 69),
 ('heard', 63),
 ('cried', 59),
 ('saw', 59),
 ('took', 56),
 ('turned', 55)]